In [8]:
import torch
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import random
import torchvision
random.seed = 421
from pathlib import Path

In [59]:
class Ageset(Dataset):

    def __init__(self, path, transforms = None, valid=False, split_pct = 0.3):
        self.image_paths = list(Path(path).rglob("*.png"))
        random.shuffle(self.image_paths)
        split_point = int(len(self)*0.3)
        if valid:
            self.image_paths = self.image_paths[:split_point]
        else:
            self.image_paths = self.image_paths[split_point:]
    def __len__(self):
        return len(self.image_paths)

    def imgpath_to_tensor(self, imgpath):
        return transforms.PILToTensor()(Image.open(imgpath)).float()

    def __getitem__(self,i):
        # return self.image_paths[i]
        if isinstance(i, slice):
            return [self[n] for n,_ in enumerate(self.image_paths[i])]
        return (self.imgpath_to_tensor(self.image_paths[i]),
                int(self.image_paths[i].parent.name))

In [10]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1 = nn.Conv2d(3,20,3)
        self.conv2 = nn.Conv2d(20,20,3)
        self.fc = nn.Linear(768320,1)
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        return self.fc(out.view(-1,768320))

In [61]:
class adaptedRes(nn.Module):
    def __init__(self):
        super(adaptedRes, self).__init__()
        resnet18 = torchvision.models.resnet18(pretrained=True)
        modules=list(resnet18.children())[:-1]
        self.resnet =nn.Sequential(*modules)
        self.fc = nn.Linear(in_features=512, out_features=1, bias=True)

        # for m in self.modules():
        # if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(self.fc.weight, gain = 1)

    def forward(self,x):
        out = self.resnet(x)
        x = torch.flatten(out, 1)
        return self.fc(x)

In [62]:
NUM_EPOCH = 10
LR = 0.001
BATCH_SIZE = 8
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_set = Ageset("data/face_age")
test_set = Ageset("data/face_age", valid=True)
train_dl = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_dl = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

model = adaptedRes()
model.to(DEVICE)
opt = torch.optim.Adam(model.parameters(), LR)

def mae_loss(y, pred):
    return torch.mean(torch.abs(y-pred))
loss_fn = mae_loss



for epoch in range(NUM_EPOCH):
    model.train()
    for data in train_dl:
        x, y = data[0].to(DEVICE), data[1].to(DEVICE)
        opt.zero_grad()
        pred = model(x)
        loss = loss_fn(y, pred)
        loss.backward()
        opt.step()
    model.eval()
    running_loss = 0
    with torch.no_grad():
        for data in test_dl:
            x, y = data[0].to(DEVICE), data[1].to(DEVICE)
            running_loss += loss_fn(y, model(x))
    print(f'loss after epoch {epoch}: {running_loss/len(test_set)})
    


tensor(1.3079)
tensor(1.1576)
tensor(1.2699)
tensor(1.1871)
tensor(1.1775)


KeyboardInterrupt: 

In [60]:
test_set = Ageset("data/face_age", valid=True)
for item, target in test_set[:10]:
    print(model(item[None]), target)

tensor([[28.5343]], grad_fn=<AddmmBackward>) 16
tensor([[28.5462]], grad_fn=<AddmmBackward>) 16
tensor([[28.5460]], grad_fn=<AddmmBackward>) 16
tensor([[28.5455]], grad_fn=<AddmmBackward>) 37
tensor([[28.4859]], grad_fn=<AddmmBackward>) 24
tensor([[28.5513]], grad_fn=<AddmmBackward>) 16
tensor([[28.5138]], grad_fn=<AddmmBackward>) 32
tensor([[28.5498]], grad_fn=<AddmmBackward>) 64
tensor([[28.5382]], grad_fn=<AddmmBackward>) 16
tensor([[28.5316]], grad_fn=<AddmmBackward>) 16


In [52]:
a = slice(0,2)
[1,2,3][a]

[1, 2]